# Top 100 종목에 대한 Naver 뉴스 수집

In [1]:
from pathlib import Path

import pandas as pd
import numpy as np

import pickle

In [10]:
import requests
from bs4 import BeautifulSoup as bs


In [45]:
from tqdm import tqdm

In [2]:
CWD = Path.cwd()
WORKSPACE_PATH = CWD.parent
COMMON_PATH = WORKSPACE_PATH / 'common'
DATA_PATH = WORKSPACE_PATH / 'data'

## 데이터 불러오기 (Top 100 returns)

In [3]:
returns_df = pd.read_pickle(DATA_PATH / 'returns_df_top100.pkl')

In [43]:
START_DATE = returns_df.index[0].strftime('%Y.%m.%d')
END_DATE = returns_df.index[-1].strftime('%Y.%m.%d')

START_DATE, END_DATE

('2020.01.02', '2024.07.12')

In [5]:
top100_tickers = returns_df.columns.to_list()

In [8]:
# with open(DATA_PATH / 'top100_tickers.pkl', 'wb') as f:
#     pickle.dump(top100_tickers, f)

## 네이버 종목뉴스 크롤링

### 뉴스 제목, 링크

In [54]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Function to scrape a single page
def scrape_page(sid, page):
    sid = str(sid).zfill(6)
    url = f'https://finance.naver.com/item/news_news.naver?code={sid}&page={page}'
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    news_list = []
    table = soup.find('table', {'class': 'type5'})
    rows = table.find_all('tr')

    for row in rows:
        # Identify main article rows
        if row.get('class') is None or 'relation_tit' in row.get('class', []):
            cols = row.find_all('td')
            if cols and cols[0].find('a', {'class': 'tit'}):
                title_tag = cols[0].find('a', {'class': 'tit'})
                if title_tag:
                    title = title_tag.text.strip()
                    href = title_tag['href']
                    if 'news_read' in href:
                        params = href.split('?')[1]
                        params_dict = dict(param.split('=') for param in params.split('&'))
                        office_id = params_dict.get('office_id')
                        article_id = params_dict.get('article_id')
                        full_link = f'https://n.news.naver.com/mnews/article/{office_id}/{article_id}'
                        info_provider = cols[1].text.strip()
                        date = cols[2].text.strip()
                        news_list.append([sid, title, date, info_provider, full_link])
        # Identify child article rows
        elif 'relation_lst' in row.get('class', []):
            sub_table = row.find('table', {'class': 'type5'})
            sub_rows = sub_table.find_all('tr')
            for sub_row in sub_rows:
                sub_cols = sub_row.find_all('td')
                if sub_cols and sub_cols[0].find('a', {'class': 'tit'}):
                    title_tag = sub_cols[0].find('a', {'class': 'tit'})
                    if title_tag:
                        title = title_tag.text.strip()
                        href = title_tag['href']
                        if 'news_read' in href:
                            params = href.split('?')[1]
                            params_dict = dict(param.split('=') for param in params.split('&'))
                            office_id = params_dict.get('office_id')
                            article_id = params_dict.get('article_id')
                            full_link = f'https://n.news.naver.com/mnews/article/{office_id}/{article_id}'
                            info_provider = sub_cols[1].text.strip()
                            date = sub_cols[2].text.strip()
                            news_list.append([sid, title, date, info_provider, full_link])
    return news_list

# Function to scrape multiple pages until a specific date
def scrape_until_date(sid, end_date):
    page = 1
    all_news = []
    last_page_content = None
    
    while True:
        news_list = scrape_page(sid, page)
        if not news_list:
            break
        
        current_page_content = str(news_list)
        if current_page_content == last_page_content:
            break

        for news in news_list:
            news_date = news[2]
            if news_date < end_date:
                return all_news
            
            all_news.append(news)
        
        last_page_content = current_page_content
        page += 1

    return all_news


In [57]:
# Example usage
sid = '007110'  # Example stock code
end_date = '2024.07.14'

In [59]:
START_DATE

'2020.01.02'

In [60]:
import time

all_news = []

for i, sid in enumerate(top100_tickers):
    sid = sid[1:]
    print(f'Scraping #{i}: {sid}...')
    
    start_time = time.time()
    all_news += scrape_until_date(sid, START_DATE)
    end_time = time.time()
    
    time_taken = end_time - start_time
    print(f'Time taken for scraping #{i}: {time_taken:.2f} seconds')

Scraping #0: 000060...
Time taken for scraping #0: 6.68 seconds
Scraping #1: 000080...
Time taken for scraping #1: 55.85 seconds
Scraping #2: 000100...
Time taken for scraping #2: 37.04 seconds
Scraping #3: 000120...
Time taken for scraping #3: 51.01 seconds
Scraping #4: 000150...
Time taken for scraping #4: 33.23 seconds
Scraping #5: 000210...
Time taken for scraping #5: 4.74 seconds
Scraping #6: 000250...
Time taken for scraping #6: 15.00 seconds
Scraping #7: 000270...
Time taken for scraping #7: 62.99 seconds
Scraping #8: 000660...
Time taken for scraping #8: 66.12 seconds
Scraping #9: 000720...
Time taken for scraping #9: 58.14 seconds
Scraping #10: 000810...
Time taken for scraping #10: 52.75 seconds
Scraping #11: 000990...
Time taken for scraping #11: 10.11 seconds
Scraping #12: 001040...
Time taken for scraping #12: 51.20 seconds
Scraping #13: 001440...
Time taken for scraping #13: 17.15 seconds
Scraping #14: 001450...
Time taken for scraping #14: 51.80 seconds
Scraping #15: 001

In [62]:
all_news_df = pd.DataFrame(all_news, columns=['sid', 'title', 'date', 'info_provider', 'full_link'])
all_news_df

,sid,title,date,info_provider,full_link
0,000060,'여의도 금융중심 계획' 결정고시 눈앞…시행사들 기다림 끝나간다,2024.07.18 21:03,이데일리,https://n.news.naver.com/mnews/article/018/000...
1,000060,"힘내요, 한 발 한 발…든든한 금융지주가 사다리를 놓아줍니다",2024.07.18 16:26,한국경제,https://n.news.naver.com/mnews/article/015/000...
2,000060,"""휴대폰 파손·항공 지연 대비""…네이버페이 여행보험 플랜 비교",2024.07.18 10:16,뉴스1,https://n.news.naver.com/mnews/article/421/000...
3,000060,"보험사 2분기 실적, 생보 웃고 손보 운다…제3보험·車보험 변수",2024.07.17 08:44,아시아경제,https://n.news.naver.com/mnews/article/277/000...
4,000060,"진격의 삼성·키움증권, 순익 2·3위로… 메리츠·NH 넘어서나?",2024.07.16 16:35,머니S,https://n.news.naver.com/mnews/article/417/000...
...,...,...,...,...,...
1237297,462870,"IPO 앞둔 시프트업, `희망퇴직·성추문 의혹` 잇단 악재",2023.07.24 16:55,디지털타임스,https://n.news.naver.com/mnews/article/029/000...
1237298,462870,"[단독] '성추문·폭언 의혹' 시프트업 투자사 대표 ""책임지고 퇴사하겠...",2023.07.24 12:53,아이뉴스24,https://n.news.naver.com/mnews/article/031/000...
1237299,462870,"시프트업 '데스티니차일드', 9월 21일 서비스 종료",2023.07.20 17:07,지디넷코리아,https://n.news.naver.com/mnews/article/092/000...
1237300,462870,"시프트업 '데스티니 차일드', 9월 21일 서비스 종료",2023.07.20 16:47,전자신문,https://n.news.naver.com/mnews/article/030/000...


In [63]:
len(top100_tickers)

190

In [65]:
all_news_df.to_pickle(DATA_PATH / 'all_news_df_top100.pkl') # 체크포인트. 200MB 넘음. 

In [67]:
all_news_df = pd.read_pickle(DATA_PATH / 'all_news_df_top100.pkl')

### 뉴스 본문 크롤링

제목만 크롤링했는데 200MB 넘는다. 본문 크롤링 시 
- 일단 request가 확 늘어남. 한 페이지당 뉴스 10개니 10배. 
- 용량도 확 늘어남. 기사 본문이 제목보다 100배 더 길다고 치면 20GB
    - 메모리 관리 위해 분할 저장하는 전략이 필요할 수 있음. every 100개마다 pickle로 떨구고 다시 가자. 
        - 일단 이렇게 해놓겠음. 
    - 그러더라도 분석을 위해선 결국 다 올려야 할 수 있음. 이 경우 다음 전략을 고려
        - 전략 1: 
            - 100개씩 나눠놓은 dataframe을 vector DB화
            - vector input으로 LLM 돌릴 수 있는 방법이 있다면 이것이 최선일 수 있음. 
            - 그러나, 우리의 UX대로 나중에 기사 원문을 보기 위해선 역 reference가 가능하도록 구현해야 함. 
                - 또는, 굳이 그럴 것 없이 href 남겨져 있으니까 기사 원문 보고 싶으면 클릭해서 네이버 뉴스로 보내는 방법도 있음. 
                - 사실 저작권을 고려하면 이렇게 구현하는 것이 맞음. 
        - 전략 2:
            - 100개 나눠놓은 dataframe을 LLM 돌려 graph RAG 연결관계 만들어내고, 
            - 각 df에서 본문 drop시키고
            - 모든 df를 concat. --> 기사 timestamp, 제목, 연결관계만 남음. 
            - Hyper CLOVA vector DB 연동 어떻게 되는지 잘 모르므로 이게 나을 수 있음. 
            - 역 reference는 위와 같은 방법으로 네이버 뉴스 href 제공. 